In [2]:
import datetime
import pandas as pd
from py2neo import Graph

In [3]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [7]:
graph = Graph("bolt://neo4j.het.io:7687")

In [8]:
query = """
MATCH (n) RETURN COUNT(n) as count_nodes
"""
graph.run(query).to_data_frame()

,count_nodes
0,47031


In [45]:
def self_describe(url, type, credentials=None):
    
    '''
    This function takes in an URL pointer to a Knowledge graph and return summary statistics 
    which can used for metadata analysis. 
    
    Args:
        url: URL pointer to KG
        type: Type of Knowledge Graph: NEO4J, GraphDB etc. 
        credentials: Local credentials file containing authorization if necessary.
    '''
    
    user, password = None, None
    
    if credentials:
        with open(credentials) as file:
            contents = file.readlines()
            user = (contents[0].split(':')[1]).strip()
            password = (contents[1].split(':')[1]).strip()
            
            
    if user and password:
        graph = Graph(url, user = user, password=password)
        query = "MATCH (n) RETURN COUNT(n) as count_nodes"
        dataframe = graph.run(query).to_data_frame()
        return dataframe
        
        
    graph = Graph(url, user = user, password=password)
    global_summary_stats_query = """MATCH (n) 
            WITH labels(n) as labels, size(keys(n)) as props, size((n)--()) as degree
            RETURN
            count(*) AS NumofNodes,
            avg(props) AS AvgNumOfPropPerNode,
            min(props) AS MinNumPropPerNode,
            max(props) AS MaxNumPropPerNode,
            avg(degree) AS AvgNumOfRelationships,
            min(degree) AS MinNumOfRelationships,
            max(degree) AS MaxNumOfRelationships
            """
    
    relationships_query = 'MATCH ()-[r]->() RETURN count(r) as count_relationships;'
    num_relationships = graph.run(relationships_query).to_series()
    
    dataframe_global = graph.run(global_summary_stats_query).to_data_frame()
    dataframe_global['NumRelationships'] = num_relationships.values
    
    
    label_wise_stats = '''MATCH (n) 
    WITH labels(n) as labels, size(keys(n)) as props, size((n)--()) as degree
    RETURN
    DISTINCT labels,
    count(*) AS NumofNodes,
    avg(props) AS AvgNumOfPropPerNode,
    min(props) AS MinNumPropPerNode,
    max(props) AS MaxNumPropPerNode,
    avg(degree) AS AvgNumOfRelationships,
    min(degree) AS MinNumOfRelationships,
    max(degree) AS MaxNumOfRelationships'''

    dataframe_local = graph.run(label_wise_stats).to_data_frame()
    return dataframe_global, dataframe_local

In [46]:
dataframe_global, dataframe_local = self_describe("bolt://neo4j.het.io:7687", type='NEO-4J')

In [48]:
dataframe_local

,labels,NumofNodes,AvgNumOfPropPerNode,MinNumPropPerNode,MaxNumPropPerNode,AvgNumOfRelationships,MinNumOfRelationships,MaxNumOfRelationships
0,[Gene],20945,6.998233,6,7,122.684030,0,9372
1,[SideEffect],5734,5.000000,5,5,24.231601,0,932
2,[BiologicalProcess],11381,5.000000,5,5,49.161234,2,999
3,[MolecularFunction],2884,5.000000,5,5,33.710818,2,955
4,[Compound],1552,7.000000,7,7,132.422036,0,1140
5,[CellularComponent],1391,5.000000,5,5,52.887132,2,1000
6,[PharmacologicClass],345,6.000000,6,6,2.982609,1,22
7,[Pathway],1822,4.161361,4,5,46.307355,2,1956
8,[Disease],137,5.000000,5,5,271.291971,0,1159
9,[Symptom],438,5.000000,5,5,7.664384,0,49


In [57]:
a = graph.run("CALL db.stats.retrieve('TOKENS');").to_data_frame()

In [63]:
a['data'].apply(pd.Series)['relationshipTypes'][0]

['EXPRESSES_AeG',
 'PARTICIPATES_GpMF',
 'REGULATES_GrG',
 'INTERACTS_GiG',
 'DOWNREGULATES_AdG',
 'PARTICIPATES_GpBP',
 'CAUSES_CcSE',
 'UPREGULATES_AuG',
 'COVARIES_GcG',
 'PARTICIPATES_GpPW',
 'DOWNREGULATES_DdG',
 'PARTICIPATES_GpCC',
 'DOWNREGULATES_CdG',
 'UPREGULATES_DuG',
 'BINDS_CbG',
 'UPREGULATES_CuG',
 'RESEMBLES_CrC',
 'LOCALIZES_DlA',
 'ASSOCIATES_DaG',
 'PRESENTS_DpS',
 'RESEMBLES_DrD',
 'TREATS_CtD',
 'INCLUDES_PCiC',
 'PALLIATES_CpD']

In [ ]:
S

In [ ]:
#### Step 1. Ru